##### Aqui está sendo importando 2 bibliotecas: gym para poder simular ambientes de RF, e random para poder criar dados randomicos.
##### Após o gym cria um ambiente chamado 'Taxi-v3', ambiente classico onde um taxi deve pegar e deixar passageiros em locais especificos, e outro parâmetro permite que a saída seja impressa no console.
##### Retorna o ambiente ao seu estado original.
##### E printa a renderização textual no console.

In [3]:
import gym  
import random  

random.seed(1234) 

streets = gym.make("Taxi-v3", render_mode = "ansi").env  
streets.reset()  
print("\n"+streets.render())  


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




Estamos definindo aonde o taxi vai começar

In [5]:
initial_state = streets.encode(2, 3, 2, 0)
streets.s = initial_state
print("\n"+streets.render())  


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




Mostrando as recompensas para cada movimento dele, sao 6 movimentos 'possíveis' as 4 direções, deixar ou pegar um passageiro, o numero maior diz a probablidade disso acontecer, o terceiro diz quantos pontos ele vai ser penalizado e o ultimo diz se houve uma pega ou deixa de passageiros sucedida.

In [7]:
streets.P[initial_state]

{0: [(1.0, 368, -1, False)],
 1: [(1.0, 168, -1, False)],
 2: [(1.0, 288, -1, False)],
 3: [(1.0, 248, -1, False)],
 4: [(1.0, 268, -10, False)],
 5: [(1.0, 268, -10, False)]}

Foi necessário fazer algumas alterações no código, pois o original não estava funcionando como deveria por causa da diferença de versões do Gym, mas basicamente o que todo esse código faz é o aprendizado por reforço 10000x (reduzi para 800) pois está com problemas para rodar

In [9]:
import numpy as np
import random


q_table = np.zeros([streets.observation_space.n, streets.action_space.n])

learning_rate = 0.1
discount_factor = 0.6
exploration = 0.1
epochs = 800

for taxi_run in range(epochs):
    state = streets.reset()
    done = False
    
    while not done:
        random_value = random.uniform(0, 1)
        if random_value < exploration:
            action = streets.action_space.sample()
        else:
            state_index = state[0] if isinstance(state, tuple) else state
            
            if isinstance(state_index, int) and 0 <= state_index < q_table.shape[0]:
                action = np.argmax(q_table[state_index])
            else:
                raise ValueError(f"Estado inválido: {state}")

        next_state, reward, done, truncated, info = streets.step(action)
        
        next_state_index = next_state[0] if isinstance(next_state, tuple) else next_state
        
        prev_q = q_table[state_index, action]
        next_max_q = np.max(q_table[next_state_index])
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
        q_table[state_index, action] = new_q

        state = next_state
    
    #adicionei uma contagem de progresso, para ter uma melhor noção.
    if taxi_run % 100 == 0:
        print(f"Progresso: {taxi_run}/{epochs}")

print("Treinamento concluído")

C:\Users\lshun\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Progresso: 0/800
Progresso: 100/800
Progresso: 200/800
Progresso: 300/800
Progresso: 400/800
Progresso: 500/800
Progresso: 600/800
Progresso: 700/800
Treinamento concluído


Isso vai nos mostrar qual é o melhor movimento a se fazer, considerando o menor valor de Q

In [12]:
q_table[initial_state]

array([-2.42893407, -2.43139767, -2.42904661, -2.42966555, -7.93080911,
       -6.28225091])

Aqui ele simula 10 viagens do táxi tentando fazer o menor caminho possível.

In [31]:
from IPython.display import clear_output
from time import sleep

for tripnum in range(1, 11):
    state = streets.reset()
   
    done = False
    
    while not done:        
        action = np.argmax(q_table[state])
        next_state, reward, done, info = streets.step(action)
        clear_output(wait=True)
        print("Trip number " + str(tripnum))
        print(streets.render(mode='ansi'))
        sleep(.5)
        state = next_state
        
    sleep(2)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices